# Polymerization simulation of grafting from a sphere

**`By You-Liang Zhu`**

## Introduction

This tutorial provides an introduction of polymerization model of GALAMOST. It is designed for new users who want to learn about how to run polymerization simulations. It does however assume that you have a machine with GALAMOST correctly installed.

In this tutorial, we would like to make a system consisting of a sphere and solvents to simulate the grafting from sphere surface.

**Download [running files](https://bitbucket.org/galamostdevelopergroup/source-code/src/master/examples/Case4-Reaction-Surface-Initiated-Polymerization/system2/)**.

In order to run this simulation, two files should be prepared:
*  `filename.xml`: file describes the information (including position, velocity, topology, etc.) of the particles. More about `XML` [data format](https://galamost.readthedocs.io/en/latest/data-format.html). The initial configuration file `filename.xml` could be generated by `Molgen` plugin.
*  `filename.gala`: file describes the settings for the simulation.

## Prepare XML file by Molgen

The coordinates of a sphere with given number of particles could be downloaded from http://neilsloane.com/icosahedral.codes/index.html. However, the raw coordinate data of a sphere has to be  processed by the following python script. This script will convert the coordinate of particles in XYZ format to be one in XML format. Moreover, the original radius of sphere defaulted as 1.0 could be adjusted and a proportion of particles are selected as initiators for polymerization by giving another type. 

The converting python script "convert-to-xml-for-sphere.py" is given:

In [ ]:
import os
import string
import random
#from __future__ import print_function
lx=30.0	     #box size
ly=30.0	
lz=30.0	
sigma=0.2    # surfacial density of initiators
radius=3     #radius of sphere
rmin=1.0     # the minimum distance between any two initiators

dat=file("icover.3.312.5.1.txt")   # downloaded from http://neilsloane.com/icosahedral.codes/index.html. 
data=[]
for line in dat:
		lin=line.strip('\n')
		data.append(float(lin))
nline=len(data)/3
pos=[]
type=[]
for j in range(0, nline):
	pos.append([data[j*3]*radius, data[j*3+1]*radius, data[j*3+2]*radius])
	type.append("A")

# distance check between initiators
npp=len(pos)
id_list=[]
def check(id):
	for i in range(0, len(id_list)):
		k=id_list[i]
		dx = pos[k][0]-pos[id][0]
		dy = pos[k][1]-pos[id][1]	
		dz = pos[k][2]-pos[id][2]
		rsq=dx*dx + dy*dy + dz*dz
		if(rsq<rmin*rmin):
			return False
	id_list.append(id)
	return True

for i in range(0, int(sigma*float(npp))):
	id=int(random.random()*float(npp))
	while (not check(id)):	
		id = int(random.random()*float(npp))
	type[id]="B"

# out put xml file
opd=open("sphere.xml","w")		
opd.write('<?xml version ="1.0" encoding ="UTF-8" ?>\n')
opd.write('<galamost_xml version="1.3">\n')
opd.write('<configuration time_step="0" dimensions="3" natoms="'+str(len(pos))+'" >\n')
opd.write('<box lx="'+str(lx)+'" ly="'+str(ly)+'" lz="'+str(lz)+'"/>\n')
opd.write('<position num="'+str(len(pos))+'">\n')
for i in range(0, len(pos)):
	opd.write(str(pos[i][0])+"  "+str(pos[i][1])+"  "+str(pos[i][2])+"\n")
opd.write('</position>\n')
opd.write('<type num="'+str(len(type))+'">\n')
for i in range(0, len(type)):
	opd.write(type[i]+"\n")
opd.write('</type>\n')
opd.write('</configuration>\n')
opd.write('</galamost_xml>\n')

After exacution of the about script, a XML file named "sphere.xml" will be generated. Further, a molgen script is employed to generate the system including the sphere and solvents by reading "sphere.xml". The initiator and active monomer are indicate by Init and Cris values.

The [`Molgen`](https://galamost.readthedocs.io/en/latest/molgen.html) script "gencon.molg" is given:

In [ ]:
#!/usr/bin/python
import sys
sys.path.append('/opt/galamost3/lib')
import molgen

lx=30.0
ly=30.0
lz=30.0
npp=312
radius=3.0
nppfile="sphere.xml"


mol0=molgen.Molecule(nppfile, npp)# filename, the number of total particles
mol0.setInit("B", 1) # 1 represents initiator; the default value is 0.
mol0.setCris("A", 1) # 0 represents reactive monomer; 1 represents inert monomer; the default value is 0.

mol1=molgen.Molecule(1) # the number of particles in a molecule
mol1.setParticleTypes("C") # particle type
mol1.setCris("C", 0) # 0 represents reactive monomer; 1 represents inert monomer; the default value is 0.
mol1.setSphere(0.0, 0.0, 0.0, radius, lz) # center position(x,y,z) of sphere, spherical shell with radius 
                                          #r_min and r_max for placing particles.

gen=molgen.Generators(lx,ly,lz) # box size
gen.addMolecule(mol0,1) # molecule type, number of molecules
gen.addMolecule(mol1,20000) # molecule type, number of molecules
gen.setMinimumDistance(0.9)# minimum distance between any two particles
gen.outPutXml("gencon") # output filename

The concentration of lipid in solution could be tuned by changing the number of the molecules.

## Prepare GALA file

GALA script "ballsip.gala" that defines the program settings is given for polymerization simulation run.

In [ ]:
#!/usr/bin/python
import sys
sys.path.append('/opt/galamost3/lib')  # the path where the GALAMOST program is installed
import galamost
from optparse import OptionParser

global _options
parser = OptionParser()
parser.add_option('--gpu', dest='gpu',help='GPU on which to execute')
(_options, args) = parser.parse_args()

filename = 'gencon.xml'# initial configuration file
build_method = galamost.XmlReader(filename)
perform_config = galamost.PerformConfig(_options.gpu)  # assign GPU by index
all_info = galamost.AllInfo(build_method,perform_config)# build system information

dt = 0.001
app = galamost.Application(all_info, dt)  # build up an application with system information and integration time-step

neighbor_list = galamost.NeighborList(all_info, 1.12246 ,0.2)#(,rcut,rbuffer)
dpdThermoLjForce = galamost.DpdThermoLjForce(all_info, neighbor_list, 1.12246 ,12371)#(,,rcut,seed for RNG)
dpdThermoLjForce.setSigma(3.0) # the parameter sigma for DPD thermostat
dpdThermoLjForce.setParams('A', 'A' ,0.0 ,1.0 ,1.0)#(type,type,epsilon,sigma,alpha)
dpdThermoLjForce.setParams('A', 'B' ,0.0 ,1.0 ,1.0)#(type,type,epsilon,sigma,alpha)
dpdThermoLjForce.setParams('A', 'C' ,1.0 ,1.0 ,1.0)#(type,type,epsilon,sigma,alpha)
dpdThermoLjForce.setParams('B', 'B' ,0.0 ,1.0 ,1.0)#(type,type,epsilon,sigma,alpha)
dpdThermoLjForce.setParams('B', 'C' ,1.0 ,1.0 ,1.0)#(type,type,epsilon,sigma,alpha)
dpdThermoLjForce.setParams('C', 'C' ,1.0 ,1.0 ,1.0)#(type,type,epsilon,sigma,alpha)
app.add(dpdThermoLjForce)

#beforehand set the type of bonds which will be generated by reaction
all_info.addBondType('polymer')

bondforcefene = galamost.BondForceFene(all_info)
bondforcefene.setParams('polymer', 10 ,1.2)#(,K0, R0)
app.add(bondforcefene)

group = galamost.ParticleSet(all_info,'C')
Gwvv = galamost.DpdGwvv(all_info, group)
app.add(Gwvv)

sort_method = galamost.Sort(all_info)
sort_method.setPeriod(300)# (period)
app.add(sort_method)

comp_info = galamost.ComputeInfo(all_info, group)

DInfo = galamost.DumpInfo(all_info, comp_info, 'data.log')
DInfo.setPeriod(500)# (period)
app.add(DInfo)

xml = galamost.XmlDump(all_info, 'p')
xml.setPeriod(10000)# (period)
xml.setOutputBond(True)
xml.setOutputInit(True)
xml.setOutputCris(True)
app.add(xml)

mol2 = galamost.Mol2Dump(all_info, 'p')
mol2.setPeriod(2000)# (period)
mol2.setChangeFreeType('C')
#mol2.deleteBoundaryBond(True)
app.add(mol2)

#ready ro run 
app.run(2000)
# Period2: change larger time step to relax
app.setDt(0.002)
app.run(8000)
# Period3: start the SIP reaction
mol2.setPeriod(2000)
reaction = galamost.Polymerization(all_info, neighbor_list, 1.12246 ,161)
reaction.setPr(0.002)
reaction.setPeriod(50)
app.add(reaction)
#DInfo.setPeriod(1)# (period)
app.run(1000000)
# Period4: relaxation after the SIP reaction
reaction.setPr(0)
mol2.setPeriod(50000)
app.run(200000)
neighbor_list.printStats()

[OVITO](http://www.ovito.org/) is suggested to render the pictures of system snapshots recorded in the output XML files. To filtrate the free monomers which are in a same particle type of polymer beads for drawing pictures, GALAMOST plugin [`galaTackle`](https://galamost.readthedocs.io/en/latest/galatackle.html), which can be find in `galamost installed path\bin`, could be employed to change the particle type of unreacted monomers by following commands. Then, a new file named filename.new.reimage.xml will be generated.

In [ ]:
galaTackle filename.xml
9:label_free_particle=C

For further calculation of properties, galaTackle also can be used, such as for the calculation of gyration radius.

In [ ]:
galaTackle *.xml
1